In [ ]:
import sys, os
sys.path.append(os.path.join(os.pardir, 'python'))

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as pl
import math
import pyvista
import ufl
import basix
import dolfinx as df
import dolfinx.fem.petsc
import utils
import sys, os
import pathlib
from mpi4py import MPI
from dolfinx import fem

In [ ]:
# define exact solution, return it (ufl and numpy representation)
def u_exact_np(x):
    U = -1
    theta = np.arctan2(x[1],x[0])

    d_psi_d_r = (U/((1/4)*(np.pi**2)-1))*((-1/4)*(np.pi**2)*(np.sin(theta))+((1/2)*(np.pi)*(theta)*(np.sin(theta)))+((theta)*(np.cos(theta))))
    d_psi_d_theta_overR = ((U)/((1/4)*(np.pi**2)-1)*(((-1/4)*(np.pi**2)*np.cos(theta))+((1/2)*(np.pi)*(np.sin(theta)))+((1/2)*(np.pi)*(theta)*(np.cos(theta)))+(np.cos(theta))-((theta)*(np.sin(theta)))))

    return np.array([(((np.cos(theta) * (d_psi_d_theta_overR))) + (np.sin(theta) * d_psi_d_r)), (((np.sin(theta) * (d_psi_d_theta_overR))) - (np.cos(theta) * d_psi_d_r))])

def u_exact(x):
    U = -1
    theta = ufl.atan2(x[1],x[0])

    d_psi_d_r = (U/((1/4)*(ufl.pi**2)-1))*((-1/4)*(ufl.pi**2)*(ufl.sin(theta))+((1/2)*(ufl.pi)*(theta)*(ufl.sin(theta)))+((theta)*(ufl.cos(theta))))
    d_psi_d_theta_overR = ((U)/((1/4)*(ufl.pi**2)-1)*(((-1/4)*(ufl.pi**2)*ufl.cos(theta))+((1/2)*(ufl.pi)*(ufl.sin(theta)))+((1/2)*(ufl.pi)*(theta)*(ufl.cos(theta)))+(ufl.cos(theta))-((theta)*(ufl.sin(theta)))))

    return ufl.as_vector([(((ufl.cos(theta) * (d_psi_d_theta_overR))) + (ufl.sin(theta) * d_psi_d_r)), (((ufl.sin(theta) * (d_psi_d_theta_overR))) - (ufl.cos(theta) * d_psi_d_r))])

In [ ]:
# define a function which returns values for Dirichlet boundary condition (60)
def u_0(x):
    u = 1
    us = u * np.ones_like(x[0])
    zeroes = np.zeros_like(x[0])
    
    return np.array([us, zeroes])

In [ ]:
def solve_batchelor(ne, p=1):
    # define the domain (unit square) of length and width ne, running in parallel
    mesh = df.mesh.create_unit_square(MPI.COMM_WORLD, ne, ne)

    # define velocity and pressure element
    v_e = basix.ufl.element("Lagrange", mesh.basix_cell(), p+1, shape=(mesh.geometry.dim,))
    p_e = basix.ufl.element("Lagrange", mesh.basix_cell(), p)

    # define mixed element
    vPe = basix.ufl.mixed_element([v_e, p_e])

    # define function space
    V = fem.functionspace(mesh, vPe)

    # define Velocity and Pressure function space
    V_v, _ = V.sub(0).collapse()
    V_p, _ = V.sub(1).collapse()

    # define test functions
    v_t, p_t = ufl.TestFunctions(V)

    # define trial functions
    v_a, p_a = ufl.TrialFunctions(V)

    # define boundary value for Dirichlet boundary condition (59)
    def boundary_D_1(x):
        return np.isclose(x[0], 0)

    dofs_D_1 = fem.locate_dofs_geometrical((V.sub(0), V_v), boundary_D_1)
    u_bc1 = fem.Function(V_v)
    u_bc1.x.array[:] = 0
    bc1 = fem.dirichletbc(u_bc1, dofs_D_1, V.sub(0))

    # define boundary value for Dirichlet boundary condition (60)
    def boundary_D_2(x):
        return np.isclose(x[1], 0)

    dofs_D_2 = fem.locate_dofs_geometrical((V.sub(0), V_v), boundary_D_2)
    u_bc2 = fem.Function(V_v)
    u_bc2.interpolate(u_0)
    bc2 = fem.dirichletbc(u_bc2, dofs_D_2, V.sub(0))

    # define boundary value for Dirichlet boundary condition (61)
    def boundary_D_3(x):
        return np.logical_or(np.isclose(x[0], 1), np.isclose(x[1], 1))

    dofs_D_3 = fem.locate_dofs_geometrical((V.sub(0), V_v), boundary_D_3)
    u_bc3 = fem.Function(V_v)
    u_bc3.interpolate(u_exact_np)
    bc3 = fem.dirichletbc(u_bc3, dofs_D_3, V.sub(0))

    def boundary_D_4(x):
        return np.logical_and(np.isclose(x[0], 0), np.isclose(x[1], 0))

    dofs_D_4 = fem.locate_dofs_geometrical((V.sub(1), V_p), boundary_D_4)
    u_bc4 = fem.Function(V_p)
    u_bc4.x.array[:] = 0
    bc4 = fem.dirichletbc(u_bc4, dofs_D_4, V.sub(1))

    # assemble weak form
    K = ufl.inner(ufl.sym(ufl.grad(v_t)), ufl.sym(ufl.grad(v_a))) * ufl.dx
    G = -ufl.div(v_t)*p_a*ufl.dx
    D = -p_t*ufl.div(v_a)*ufl.dx
    S = K + G + D

    zero = fem.Constant(mesh, df.default_scalar_type(0.0))
    L = zero*p_t*ufl.dx

    # assemble problem solver and solve
    problem = df.fem.petsc.LinearProblem(S, L, bcs=[bc1, bc2, bc3, bc4], petsc_options={"ksp_type": "preonly", "pc_type": "lu", "pc_factor_mat_solver_type": "mumps"})
    uh = problem.solve()

    vh = uh.sub(0).collapse()

    return vh

In [ ]:
def error(vh):
    x = ufl.SpatialCoordinate(vh.function_space.mesh)
    vex = u_exact(x)
    error_L2 = fem.assemble_scalar(fem.form((vh - vex)**2 * ufl.dx))
    error_L2 = np.sqrt(MPI.COMM_WORLD.allreduce(error_L2, op=MPI.SUM))
    
    return error_L2

In [ ]:
def convergence():
    fig = pl.figure() # open plotting figure

    # Make an output folder
    output_folder = pathlib.Path("output")
    output_folder.mkdir(exist_ok=True, parents=True)
    
    ax = fig.gca()
    
    ps = [1, 2] # polynomial orders to try

    nelements = [10, 20, 40, 80] # resolutions to try

    test_passes = True # expected order of convergence flag

    for p in ps: # loop over polynomial orders
        errors_l2_a = [] # accumulate l2 errors
        for ne in nelements: # loop over resolutions
            vh = solve_batchelor(ne, p) # solve 2D Poisson
            errorl2 = error(vh) # evalute approximate solution error
            print('ne = ', ne, ', l2error = ', errorl2) # print and save l2 error
            errors_l2_a.append(errorl2)
            
        hs = 1./np.array(nelements)/p #work out order of convergence

        # Write the errors to disk
        with open(output_folder / 'batchelor_cornerflow{}.csv'.format(p), 'w') as f:
            np.savetxt(f, np.c_[nelements, hs, errors_l2_a], delimiter=',', 
                   header='nelements, hs, l2errs')
        
        fit = np.polyfit(np.log(hs), np.log(errors_l2_a),1) # fit line to convergence data for l2 error
        print("***********  order of accuracy p={}, order={}".format(p,fit[0]))

        # log-log plot of the L2 error 
        ax.loglog(hs,errors_l2_a,'o-',label='p={}, order={:.2f}'.format(p,fit[0]))
    
        # Test if the order of convergence is as expected
        test_passes = test_passes and abs(fit[0]-1) < 0.1

    # Tidy up the plot
    %matplotlib inline
    ax.set_xlabel('h')
    ax.set_ylabel('||e||_2')
    ax.grid()
    ax.set_title('Convergence: L2 Error')
    ax.legend()
    fig.savefig('batchelor_cornerflow.pdf')
    fig.show()

    pl.savefig(output_folder / 'batchelor.pdf')
    
    print("***********  convergence figure in batchelor.pdf")
    # Check if we passed the test
    assert(test_passes)

In [ ]:
# main
solution = solve_batchelor(4)
#visualize(solution)
utils.plot_vector(solution, glyph_factor=0.3)
error1 = error(solution)
print("L2 error:")
print(error1)
convergence()